In [10]:
from pathlib import Path # type: ignore
from freyja_plot import FreyjaPlotter
import plotly.io as pio
import sys
if (module_path:=str(Path(".").absolute().resolve().parent)) not in sys.path:
    sys.path.insert(0, module_path)
from sample_info import colormap, summary_dict, expected, artic_runs as runs, plotting_dir

In [2]:
outdir = plotting_dir / "percent_of_expected/lineage_box_plots"
outdir.mkdir(exist_ok=True)

In [6]:
def sort_key(series):
    # sorted_lineages = {lin:i for i,lin in enumerate(("Wuhan-hu-1","Alpha","Gamma","Delta","Iota","BA.1.X","BA.2.X","BG.X","BA.4.X","BA.5.X"))}
    sorted_lineages = {lin:i for i,lin in enumerate((
        "Wuhan-hu-1","Alpha","Gamma","Delta","Iota","BA.1.X","BA.2.X","BG.X","BA.4.X","BA.5.X",
        "Freyja","Alcov","VaQuERo","LCS","kallisto (C-WAP)","LolliPop","kallisto","Kraken 2 (C-WAP)","lineagespot",
    ))}
    return [sorted_lineages[lin] for lin in series]

def getPercentExpectedPlot(mix_type, fn=None):
    file_map = {expected:"Expected"}
    for tool, files in runs.items():
    # for name,file in files.items():
        file_map[files[mix_type]] = tool
    plotter = FreyjaPlotter(
        file_map=file_map,
        colormap=colormap,
        summary_dict=summary_dict,
    )
    # NOTE: Figures in paper, section 3.5, Figure 4 a-c
    print(f"Creating {mix_type} summarized boxplot")
    fig = plotter.plotPercentExpectedBox(summarized=True,sort_key=sort_key,title=f"Lineage abundance estimates as a percent of expected abundance - {mix_type} mixtures")
    return fig


In [ ]:
figs = {}

In [35]:
# create or update figs
for mix_type in ("WB","NWRB","PWRB"):
    fig = figs.get(mix_type,None)
    if fig is None:
        fig = getPercentExpectedPlot(mix_type)
    fig.update_yaxes(range=[0,460])
    figs[mix_type] = fig

In [39]:
# write out and display figs (Figure 4a-c in paper)
for mix_type in ("WB","NWRB","PWRB"):
    fn=outdir/f"{mix_type}-percent-expected-box-summary.png"
    fig = figs[mix_type]
    pio.write_image(fig, fn, width=1200, height=700, scale=2)
    fig.show()